<a href="https://colab.research.google.com/github/jessilver/Organizacao_Computadores/blob/develop/Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Faça uma cópia no seu drive

Importante: as instalações precisam ser feita toda vez que for iniciada uma nova seção no Colab

# Jessé Eliseu Nunes da Silva

# Como usar:

As implementações foram separadas e partes, para que a interface reconheça a parte que desesja utilizar bastas "ativa-la", trocando o valor da variavel de status de false para true

# Interface

In [4]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd

# Widgets
code_area = widgets.Textarea(
    value="",
    placeholder="Escreva seu código assembly aqui...",
    description="Code:",
    layout=widgets.Layout(width="100%", height="200px")
)

run_button = widgets.Button(description="Run Program", button_style="success")
reg_button = widgets.Button(description="Show Registers", button_style="info")
mem_button = widgets.Button(description="Show Memory", button_style="info")

start_addr = widgets.IntText(value=0, description="Start Addr")
mem_range = widgets.IntText(value=10, description="Range")

output_area = widgets.Output()


# Funções de exibição
def show_registers(regs):
    data = {"Reg": [f"x{i}" for i in range(32)],
            "Valor": [to_signed32(regs.read(i)) for i in range(32)]}
    df = pd.DataFrame(data)
    with output_area:
        display(df)


def show_memory(mem, start=0, rng=10):
    data = {"Endereço": list(range(start, start+rng)),
            "Valor": [mem.load_word(i) for i in range(start, start+rng)]}
    df = pd.DataFrame(data)
    with output_area:
        display(df)

# === Função para rodar um programa inteiro ===
def run_program(asm_code, regs, mem):
    program = parse_program(asm_code)
    cycle = 0
    for instr in program:
        cycle += 1

        try:
          if part_1_status == True:
            info = execute_type_1_instruction(instr, regs, mem)
        except Exception as e:
          pass

        print(f"Ciclo {cycle}: {instr} -> {info}")
    return regs, mem

# Handlers dos botões
def run_program_handler(b):
    clear_output(wait=True)
    global regs, mem
    regs = RegisterFile()
    mem = SimpleMemory(64)

    code = code_area.value
    run_program(code, regs, mem)

    with output_area:
        print("Programa executado.")


def parse_program(asm_code):
    program = []
    for line in asm_code.strip().splitlines():
        line = line.strip()
        if not line or line.startswith("#"):
            continue  # ignora linha vazia/comentário

        parts = line.replace(",", "").split()
        op = parts[0].upper()

        if op in ["add", "sub", "mul"]:
            rd = int(parts[1][1:])   # remove 'x' -> inteiro
            rs1 = int(parts[2][1:])
            rs2 = int(parts[3][1:])
            instr = {"op": op, "rd": rd, "rs1": rs1, "rs2": rs2}

        elif op == "lw":
            rd = int(parts[1][1:])
            addr = int(parts[2])
            instr = {"op": op, "rd": rd, "addr": addr}

        elif op == "sw":
            rs1 = int(parts[1][1:])
            addr = int(parts[2])
            instr = {"op": op, "rs1": rs1, "addr": addr}

        else:
            raise ValueError(f"Instrução desconhecida: {line}")

        program.append(instr)
    return program

def reg_button_handler(b):
    clear_output(wait=True)
    show_registers(regs)


def mem_button_handler(b):
    clear_output(wait=True)
    show_memory(mem, start=start_addr.value, rng=mem_range.value)

# Simple Memory Class
class SimpleMemory:
    def __init__(self, size_in_words):
        self.size = size_in_words
        self.mem = [0] * size_in_words

    def load_word(self, address):
        if 0 <= address < self.size:
            return self.mem[address]
        else:
            raise IndexError(f"Endereço de memória inválido: {address}")

    def store_word(self, address, value):
        if 0 <= address < self.size:
            self.mem[address] = value
        else:
            raise IndexError(f"Endereço de memória inválido: {address}")

# Register File Class
class RegisterFile:
    def __init__(self):
        self.regs = [0] * 32  # 32 registers

    def read(self, reg_num):
        if 0 <= reg_num < 32:
            return self.regs[reg_num]
        else:
            raise IndexError(f"Número de registrador inválido: x{reg_num}")

    def write(self, reg_num, value):
        if 0 <= reg_num < 32:
            self.regs[reg_num] = value
        else:
            raise IndexError(f"Número de registrador inválido: x{reg_num}")

# Convert 32-bit integer to signed
def to_signed32(value):
    # Assuming value is a Python integer
    if value & (1 << 31):
        return value - (1 << 32)
    return value


# Liga botões aos handlers
run_button.on_click(run_program_handler)
reg_button.on_click(reg_button_handler)
mem_button.on_click(mem_button_handler)


# Layout
ui = widgets.VBox([
    code_area,
    widgets.HBox([run_button, reg_button, start_addr, mem_range, mem_button]),
    output_area
])

display(ui)


# Parte 1


In [2]:

part_1_status = True

def execute_type_1_instruction(instr, regs, mem):
    op = instr["op"]
    info = f"Executing {op}"

    if op == "add":
        rd = instr["rd"]
        rs1 = instr["rs1"]
        rs2 = instr["rs2"]
        regs.write(rd, regs.read(rs1) + regs.read(rs2))
        info += f": x{rd} = x{rs1} + x{rs2}"

    elif op == "sub":
        rd = instr["rd"]
        rs1 = instr["rs1"]
        rs2 = instr["rs2"]
        regs.write(rd, regs.read(rs1) - regs.read(rs2))
        info += f": x{rd} = x{rs1} - x{rs2}"

    elif op == "mul":
        rd = instr["rd"]
        rs1 = instr["rs1"]
        rs2 = instr["rs2"]
        regs.write(rd, regs.read(rs1) * regs.read(rs2))
        info += f": x{rd} = x{rs1} * x{rs2}"

    elif op == "lw":
        rd = instr["rd"]
        addr = instr["addr"]
        regs.write(rd, mem.load_word(addr))
        info += f": x{rd} = mem[{addr}]"

    elif op == "sw":
        rs1 = instr["rs1"]
        addr = instr["addr"]
        mem.store_word(addr, regs.read(rs1))
        info += f": mem[{addr}] = x{rs1}"

    else:
        info += f": Unknown instruction {op}"

    return info


# # === Exemplo de uso ===
# mem = SimpleMemory(64)
# regs = RegisterFile()

# # inicializa registradores e memória
# regs.write(3, 7)
# regs.write(5, 2)
# mem.store_word(10, 123)

# codigo = """
# LOAD x1, 10
# ADD  x2, x1, x3
# MUL  x4, x2, x5
# STORE x4, 20
# """

# regs, mem = run_program(codigo, regs, mem)

# print("\nRegistradores finais:")
# for i in range(6):
#     print(f"x{i} = {to_signed32(regs.read(i))}")

# print("Mem[20] =", mem.load_word(20))

# Parte 2

# Parte 3

# Parte 4

# Parte 5